In [1]:
import time

from gurobipy import GRB
import numpy as np
import torch
from torch.utils.data import Dataset
from tqdm import tqdm

from pyepo.model.opt import optModel


class optDatasetConstrs(Dataset):
    """
    This class is Torch Dataset for optimization problems with active constraints.

    Attributes:
        model (optModel): Optimization models
        feats (np.ndarray): Data features
        costs (np.ndarray): Cost vectors
        sols (np.ndarray): Optimal solutions
        objs (np.ndarray): Optimal objective values
        ctrs (list(np.ndarray)): active constraints
    """

    def __init__(self, model, feats, costs):
        """
        A method to create a optDataset from optModel

        Args:
            model (optModel): an instance of optModel
            feats (np.ndarray): data features
            costs (np.ndarray): costs of objective function
        """
        if not isinstance(model, optModel):
            raise TypeError("arg model is not an optModel")
        self.model = model
        # data
        self.feats = feats
        self.costs = costs
        # find optimal solutions
        self.sols, self.objs, self.ctrs = self._getSols()

    def _getSols(self):
        """
        A method to get optimal solutions for all cost vectors
        """
        sols, objs, ctrs = [], [], []
        print("Optimizing for optDataset...")
        time.sleep(1)
        for c in tqdm(self.costs):
            try:
                sol, obj = self._solve(c)
                constrs = self._getBindingConstrs()
            except:
                raise ValueError(
                    "For optModel, the method 'solve' should return solution vector and objective value."
                )
            sols.append(sol)
            objs.append([obj])
            ctrs.append(np.array(constrs))
        return np.array(sols), np.array(objs), ctrs

    def _solve(self, cost):
        """
        A method to solve optimization problem to get an optimal solution with given cost

        Args:
            cost (np.ndarray): cost of objective function

        Returns:
            tuple: optimal solution (np.ndarray) and objective value (float)
        """
        self.model.setObj(cost)
        sol, obj = self.model.solve()
        return sol, obj
    
    def _getBindingConstrs(self):
        """
        A method to get active constraints with current optimal solution

        Returns:
            np.ndarray: normal vector of constraints
        """
        constrs = []
        # iterate all constraints
        for constr in self.model._model.getConstrs():
            # check tight constraints
            if abs(constr.Slack) < 1e-5:
                t_constr = []
                # get coefficients
                for i in self.model.x:
                    t_constr.append(self.model._model.getCoeff(constr, self.model.x[i]))
                # get coefficients in standard form
                if constr.sense == GRB.LESS_EQUAL:
                    # <=
                    constrs.append(t_constr)
                elif constr.sense == GRB.GREATER_EQUAL:
                    # >=
                    constrs.append([- coef for coef in t_constr])
                elif constr.sense == GRB.EQUAL:
                    # ==
                    constrs.append(t_constr)
                    constrs.append([- coef for coef in t_constr])
                else:
                    # invalid sense
                    raise ValueError("Invalid constraint sense.")
        # iterate all constraints
        for i, v in enumerate(self.model._model.getVars()):
            t_constr = [0] * len(self.model.x)
            # check variables on bounds
            if v.x <= 1e-5:
                # x_i >= 0
                t_constr[i] = -1
                constrs.append(t_constr)
            elif v.x >= 1 - 1e-5:
                # x_i <= 1
                t_constr[i] = 1
                constrs.append(t_constr)
        return constrs
                
    def __len__(self):
        """
        A method to get data size

        Returns:
            int: the number of optimization problems
        """
        return len(self.costs)

    def __getitem__(self, index):
        """
        A method to retrieve data

        Args:
            index (int): data index

        Returns:
            tuple: data features (torch.tensor), costs (torch.tensor), optimal solutions (torch.tensor) and objective values (torch.tensor)
        """
        return (
            torch.FloatTensor(self.feats[index]),
            torch.FloatTensor(self.costs[index]),
            torch.FloatTensor(self.sols[index]),
            torch.FloatTensor(self.objs[index]),
            torch.FloatTensor(self.ctrs[index])
        )

Auto-Sklearn cannot be imported.


## Data Set and Optimization Solver 

In [2]:
import pyepo

In [3]:
# generate data
grid = (2,2) # grid size
num_data = 1000 # number of training data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.shortestpath.genData(num_data+1000, num_feat, grid, deg, e, seed=42)

In [4]:
from pyepo.model.grb import shortestPathModel
# set solver
optmodel = shortestPathModel(grid)
# test
optmodel.setObj(costs[0])
sol, obj = optmodel.solve()
print("Obj: {}".format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Obj: 0.5009689961416153
(0, 2)
(2, 3)


In [5]:
# split data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=1000, random_state=42)
# get training and test data set
dataset_train = optDatasetConstrs(optmodel, x_train[:10], c_train[:10])

Optimizing for optDataset...


100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2075.26it/s]


In [6]:
for i in range(len(dataset_train)):
    ctr = dataset_train.ctrs[i]
    sol = dataset_train.sols[i]
    print("Sol:")
    print(sol)
    print("Num of tight constr:")
    print(len(ctr))
    print("Tight constr:")
    for c in ctr:
        print(c)
    print()

Sol:
[0. 1. 0. 1.]
Num of tight constr:
12
Tight constr:
[-1. -1.  0.  0.]
[ 1.  1. -0. -0.]
[ 1.  0. -1.  0.]
[-1. -0.  1. -0.]
[ 0.  1.  0. -1.]
[-0. -1. -0.  1.]
[0. 0. 1. 1.]
[-0. -0. -1. -1.]
[-1.  0.  0.  0.]
[0. 1. 0. 0.]
[ 0.  0. -1.  0.]
[0. 0. 0. 1.]

Sol:
[1. 0. 1. 0.]
Num of tight constr:
12
Tight constr:
[-1. -1.  0.  0.]
[ 1.  1. -0. -0.]
[ 1.  0. -1.  0.]
[-1. -0.  1. -0.]
[ 0.  1.  0. -1.]
[-0. -1. -0.  1.]
[0. 0. 1. 1.]
[-0. -0. -1. -1.]
[1. 0. 0. 0.]
[ 0. -1.  0.  0.]
[0. 0. 1. 0.]
[ 0.  0.  0. -1.]

Sol:
[1. 0. 1. 0.]
Num of tight constr:
12
Tight constr:
[-1. -1.  0.  0.]
[ 1.  1. -0. -0.]
[ 1.  0. -1.  0.]
[-1. -0.  1. -0.]
[ 0.  1.  0. -1.]
[-0. -1. -0.  1.]
[0. 0. 1. 1.]
[-0. -0. -1. -1.]
[1. 0. 0. 0.]
[ 0. -1.  0.  0.]
[0. 0. 1. 0.]
[ 0.  0.  0. -1.]

Sol:
[1. 0. 1. 0.]
Num of tight constr:
12
Tight constr:
[-1. -1.  0.  0.]
[ 1.  1. -0. -0.]
[ 1.  0. -1.  0.]
[-1. -0.  1. -0.]
[ 0.  1.  0. -1.]
[-0. -1. -0.  1.]
[0. 0. 1. 1.]
[-0. -0. -1. -1.]
[1. 0. 0. 0.]
[ 0